## 1. Easy OCR


In [1]:
import os
import pandas as pd
import easyocr

# === CONFIG ===
image_folder = r"C:\projecteve\OCR\data\yolo_crop\output_detail_only2025-11-04"
output_csv = os.path.join(image_folder, "ocr_output_train_CJ.csv")

# === สร้าง EasyOCR Reader ===
reader = easyocr.Reader(['th', 'en'])

# === เตรียมไฟล์ภาพ ===
all_images = sorted([
    f for f in os.listdir(image_folder)
    if f.lower().endswith(('.jpg', '.jpeg', '.png'))
])

total = len(all_images)
ocr_results = []

print(f"📂 พบภาพทั้งหมด {total} ไฟล์ กำลังเริ่ม OCR ...")

# === วนลูปทำ OCR ทีละภาพ ===
for idx, filename in enumerate(all_images, start=1):
    image_path = os.path.join(image_folder, filename)

    print(f"📄 [{idx}/{total}] กำลังอ่าน: {filename}")

    try:
        # อ่านข้อความจากภาพ
        result = reader.readtext(image_path, detail=0)
        combined_text = '|'.join(result).strip()

        # เพิ่มเข้า list
        ocr_results.append({'filename': filename, 'text': combined_text})

    except Exception as e:
        print(f"⚠️ อ่านภาพ {filename} ไม่สำเร็จ: {e}")

# === บันทึกผลเป็น CSV ===
df = pd.DataFrame(ocr_results)
df.to_csv(output_csv, index=False, encoding='utf-8-sig')

print(f"\n✅ บันทึกผล OCR แล้ว: {output_csv}")
print(f"📊 อ่านสำเร็จ {len(ocr_results)}/{total} ไฟล์")


c:\Miniconda3\envs\donut-th\lib\site-packages\easyocr\detection.py:85: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  net.load_state_dict(copyStateDict(torch.load(trained_mod

📂 พบภาพทั้งหมด 275 ไฟล์ กำลังเริ่ม OCR ...
📄 [1/275] กำลังอ่าน: 2025-11-04_detail_0001.jpg
📄 [2/275] กำลังอ่าน: 2025-11-04_detail_0002.jpg
📄 [3/275] กำลังอ่าน: 2025-11-04_detail_0003.jpg
📄 [4/275] กำลังอ่าน: 2025-11-04_detail_0004.jpg
📄 [5/275] กำลังอ่าน: 2025-11-04_detail_0005.jpg
📄 [6/275] กำลังอ่าน: 2025-11-04_detail_0006.jpg
📄 [7/275] กำลังอ่าน: 2025-11-04_detail_0007.jpg
📄 [8/275] กำลังอ่าน: 2025-11-04_detail_0008.jpg
📄 [9/275] กำลังอ่าน: 2025-11-04_detail_0009.jpg
📄 [10/275] กำลังอ่าน: 2025-11-04_detail_0010.jpg
📄 [11/275] กำลังอ่าน: 2025-11-04_detail_0011.jpg
📄 [12/275] กำลังอ่าน: 2025-11-04_detail_0012.jpg
📄 [13/275] กำลังอ่าน: 2025-11-04_detail_0013.jpg
📄 [14/275] กำลังอ่าน: 2025-11-04_detail_0014.jpg
📄 [15/275] กำลังอ่าน: 2025-11-04_detail_0015.jpg
📄 [16/275] กำลังอ่าน: 2025-11-04_detail_0016.jpg
📄 [17/275] กำลังอ่าน: 2025-11-04_detail_0017.jpg
📄 [18/275] กำลังอ่าน: 2025-11-04_detail_0018.jpg
📄 [19/275] กำลังอ่าน: 2025-11-04_detail_0019.jpg
📄 [20/275] กำลังอ่าน: 2025-11-04_de

## LLM gemini

In [ ]:
# =========================
# Gemini OCR (VERBATIM) for Jupyter
# =========================

# !pip -q install google-genai pillow tqdm pandas

# ---------- SETTINGS ----------
import os
from pathlib import Path

# 🔑 ใส่ API keys หลายตัวได้ (แนะนำคนละโปรเจ็กต์/โควต้า)
KEY_POOL = [
    "Press Key Here",
    "Press Key Here",
]
assert len([k for k in KEY_POOL if k.strip()]) > 0, "กรุณาใส่ API key อย่างน้อย 1 ค่า"
# 📁 Paths (แก้ให้ตรงเครื่อง)
INPUT_DIR   = Path(r"C:\projecteve\OCR\data\yolo_crop\output_detail_only2025-11-04")
OUT_JSONL   = Path(r"C:\projecteve\OCR\data\yolo_crop\OCR_LLM\ocr_output_2025-11-04.jsonl")
OCR_TXT_DIR = Path(r"C:\projecteve\OCR\data\yolo_crop\OCR_LLM\ocr_txt_2025-11-04")
CHECKPOINT  = Path(r"C:\projecteve\OCR\data\yolo_crop\OCR_LLM\ocr_done_list_2025-11-04.txt")

# ⚙️ Config (โหมดคิดน้อยสุด)
MODEL             = "gemini-2.5-flash"
LANG              = "th"                 # "th" | "en"
INLINE_LIMIT_MB   = 18.0
THINKING_BUDGET   = 0
TEMPERATURE       = 0
MAX_OUTPUT_TOKENS = 8192
INIT_BATCH_SIZE   = 5
THROTTLE_SEC      = 1.5
MAX_RETRIES       = 8
MAX_IMAGES        = 0    # 0=ทั้งหมด, >0=จำกัดจำนวนเพื่อทดสอบ
EXPORT_CSV        = True

# การจัดการ newline ในผลลัพธ์: "keep" | "escape" | "one_line"
NORMALIZE_TEXT_MODE = "keep"

# ---------- IMPORTS ----------
import io, json, time, re, random
from typing import List, Tuple, Any, Dict, Set
from datetime import datetime
from PIL import Image, ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

from tqdm import tqdm
import pandas as pd

from google import genai
from google.genai import types

# ---------- FILE UTILS ----------
IMG_EXTS = {".jpg", ".jpeg", ".png", ".webp"}

def list_images(input_dir: Path) -> List[Path]:
    files = []
    for ext in IMG_EXTS:
        files.extend(sorted(input_dir.rglob(f"*{ext}")))
    return files

def ensure_dir(p: Path):
    p.mkdir(parents=True, exist_ok=True)

def _infer_mime(suffix: str) -> str:
    s = suffix.lower().lstrip(".")
    return {"png":"image/png","jpg":"image/jpeg","jpeg":"image/jpeg","webp":"image/webp"}.get(s,"application/octet-stream")

def use_inline(p: Path, limit_mb=18.0) -> bool:
    try:
        return (p.stat().st_size / (1024*1024)) <= limit_mb
    except:
        return True

def pillow_or_bytes(p: Path) -> Tuple[Any, bytes]:
    with open(p,"rb") as f: b=f.read()
    im = Image.open(io.BytesIO(b))
    return im, b

def load_done(path: Path) -> Set[str]:
    if not path.exists(): return set()
    return {ln.strip() for ln in open(path, encoding="utf-8") if ln.strip()}

def append_done(path: Path, names: List[str]):
    with open(path, "a", encoding="utf-8") as f:
        for n in names: f.write(n+"\n")

# ---------- KEY ROTATION ----------
class KeyRotator:
    def __init__(self, keys: list[str]):
        self.keys = [k.strip() for k in keys if k and k.strip()]
        if not self.keys:
            raise ValueError("KEY_POOL is empty")
        self.idx = 0
        self._client = None
        self._init_client()

    @property
    def current_key(self) -> str:
        return self.keys[self.idx]

    def _init_client(self):
        os.environ["GEMINI_API_KEY"] = self.current_key
        self._client = genai.Client(api_key=self.current_key)

    def get_client(self):
        return self._client

    def rotate(self) -> bool:
        nxt = (self.idx + 1) % len(self.keys)
        if nxt == self.idx:  # มีคีย์เดียว
            return False
        self.idx = nxt
        print(f"🔑 Switching API key → {self.idx+1}/{len(self.keys)}")
        self._init_client()
        return True

rotator = KeyRotator(KEY_POOL)
def build_client() -> "genai.Client":
    return rotator.get_client()

# ---------- JSON PARSER (ทนทาน) ----------
CONTROL_CHARS_RE = re.compile(r'[\x00-\x08\x0B\x0C\x0E-\x1F]')

def safe_json_array_parse(text: str) -> list[dict]:
    if not isinstance(text, str) or not text.strip():
        raise json.JSONDecodeError("Empty response", "", 0)

    if "```" in text:
        text = max(text.split("```"), key=len)

    text = CONTROL_CHARS_RE.sub("", text).strip()

    # 1) ทั้งก้อน
    try:
        obj = json.loads(text)
        if isinstance(obj, list): return obj
    except: pass

    # 2) ดึงบล็อก [ ... ]
    m = re.search(r'\[\s*{.*}\s*\]', text, flags=re.DOTALL)
    if m:
        try:
            obj = json.loads(m.group(0))
            if isinstance(obj, list): return obj
        except: pass

    # 3) Fallback JSONL
    arr = []
    for line in text.splitlines():
        line = CONTROL_CHARS_RE.sub("", line.strip().strip("`"))
        if not line or not (line.startswith("{") and line.endswith("}")):
            continue
        try:
            arr.append(json.loads(line))
        except: 
            continue
    if arr: return arr

    raise json.JSONDecodeError("Unable to parse JSON array from model output", text[:200] + "...", 0)

# ---------- PROMPTS (VERBATIM) ----------
def single_image_prompt(lang: str) -> str:
    if lang == "th":
        return (
            "โหมด VERBATIM OCR:\n"
            "- ถอดเฉพาะตัวอักษรที่เห็นจากภาพแบบตรงตัวเท่านั้น\n"
            "- ห้ามแปล ห้ามเดา ห้ามเติมคำ ห้ามแก้สะกด ห้ามสรุป ห้ามเรียงใหม่\n"
            "- คงบรรทัดและช่องว่างตามที่เห็นในภาพ\n"
            "- เก็บตัวเลข สัญลักษณ์ เครื่องหมาย ทุกตัวตามที่เห็น\n"
            "- ถ้าอ่านไม่ชัด ให้พิมพ์ '�' แทน ห้ามเดา\n"
            "- ตอบเป็นข้อความล้วน (plain text) เท่านั้น"
        )
    else:
        return (
            "VERBATIM OCR mode:\n"
            "- Transcribe ONLY characters visible in the image.\n"
            "- No translation, inference, normalization, correction, summarization, or reordering.\n"
            "- Preserve line breaks and spacing exactly.\n"
            "- Keep digits and symbols exactly as seen.\n"
            "- If unreadable, output '�' (do not guess).\n"
            "- Output plain text only."
        )

def make_batch_prompt_json_array(lang: str, filenames: list[str]) -> str:
    names = "\n".join(filenames)
    n = len(filenames)
    if lang == "th":
        return (
            "โหมด VERBATIM OCR (หลายไฟล์):\n"
            "- ถอดตัวอักษรตรงตัวเท่านั้น ห้ามแปล/เดา/เติม/แก้/สรุป/จัดรูป\n"
            "- คงบรรทัดและช่องว่างตามที่เห็นในภาพ\n"
            "- ตัวอ่านไม่ชัดให้ใส่ '�'\n"
            f"- ตอบเป็น **JSON Array เท่านั้น** จำนวน {n} ชิ้น เรียงตามรายชื่อด้านล่าง\n"
            'รูปแบบ: [{"image":"<filename>","text":"<verbatim-text>"} , ...]\n'
            "รายชื่อไฟล์ตามลำดับ:\n" + names + "\n"
        )
    else:
        return (
            "VERBATIM OCR (multi-file):\n"
            "- Transcribe exactly; no translation/inference/correction/summarization/reformatting.\n"
            "- Preserve line breaks and spacing.\n"
            "- Use '�' where unreadable.\n"
            f"- Reply as **JSON Array only**, length={n}, in the same order below:\n"
            '[{"image":"<filename>","text":"<verbatim-text>"} , ...]\n'
            "Filenames:\n" + names + "\n"
        )

# ---------- MODEL CALLS ----------
from google.genai import types

def call_batch_json_array(client: "genai.Client", model: str, parts: list,
                          thinking_budget: int, max_output_tokens: int, temperature: float) -> str:
    cfg = types.GenerateContentConfig(
        thinking_config=types.ThinkingConfig(thinking_budget=thinking_budget),
        max_output_tokens=max_output_tokens,
        response_mime_type="application/json",
        temperature=temperature,
    )
    resp = client.models.generate_content(model=model, contents=parts, config=cfg)
    return (resp.text or "").strip()

def process_batch_json_array(client: "genai.Client", model: str, batch_imgs: list[Path], lang: str,
                             inline_limit_mb: float, thinking_budget: int, max_output_tokens: int, temperature: float) -> tuple[list[dict], bool]:
    filenames = [p.name for p in batch_imgs]
    prompt = make_batch_prompt_json_array(lang, filenames)

    parts = [prompt]
    for p in batch_imgs:
        if use_inline(p, inline_limit_mb):
            _, b = pillow_or_bytes(p)
            parts.append(types.Part.from_bytes(data=b, mime_type=_infer_mime(p.suffix)))
        else:
            up = client.files.upload(file=str(p))
            parts.append(up)

    raw = call_batch_json_array(client, model, parts, thinking_budget, max_output_tokens, temperature)

    try:
        arr = safe_json_array_parse(raw)
        by_name = { (item.get("image") or "").strip(): (item.get("text") or "") for item in arr if isinstance(item, dict) }
        ordered = [ {"image": fn, "text": by_name.get(fn, "")} for fn in filenames ]
        ok = len(ordered) == len(filenames) and all(ordered[i]["image"] == filenames[i] for i in range(len(filenames)))
        return ordered, ok
    except Exception:
        try:
            log_dir = Path("./_gemini_raw"); log_dir.mkdir(exist_ok=True)
            ts = datetime.now().strftime("%Y%m%d_%H%M%S")
            (log_dir / f"batch_parse_error_{ts}.txt").write_text(raw, encoding="utf-8")
        except:
            pass
        ordered = [ {"image": fn, "text": ""} for fn in filenames ]
        return ordered, False

def process_single_image(client: "genai.Client", model: str, p: Path, lang: str,
                         inline_limit_mb: float, thinking_budget: int, max_output_tokens: int, temperature: float) -> str:
    prompt = single_image_prompt(lang)
    if use_inline(p, inline_limit_mb):
        _, b = pillow_or_bytes(p)
        image_part = types.Part.from_bytes(data=b, mime_type=_infer_mime(p.suffix))
    else:
        image_part = client.files.upload(file=str(p))
    cfg = types.GenerateContentConfig(
        thinking_config=types.ThinkingConfig(thinking_budget=thinking_budget),
        max_output_tokens=max_output_tokens,
        response_mime_type="text/plain",
        temperature=temperature,
    )
    resp = client.models.generate_content(model=model, contents=[prompt, image_part], config=cfg)
    return (resp.text or "").strip()

# ---------- ERROR HELPERS ----------
def _is_quota_or_rate_error(msg: str) -> bool:
    msg = (msg or "").lower()
    needles = ["429", "quota", "rate", "too many requests", "exceeded", "resourceexhausted"]
    return any(n in msg for n in needles)

def _is_transient_server_error(msg: str) -> bool:
    msg = (msg or "").lower()
    needles = ["503", "unavailable", "internal", "5xx", "server error",
               "deadline", "gateway", "temporarily", "upstream", "rst"]
    return any(n in msg for n in needles)

# ---------- SAFE WRAPPERS ----------
def safe_process_batch_json_array(*args, max_retries=MAX_RETRIES, throttle_sec=THROTTLE_SEC, **kwargs):
    base_delay = float(throttle_sec) if throttle_sec else 0.0
    delay = base_delay
    for attempt in range(max_retries):
        try:
            if attempt == 0 and delay > 0:
                time.sleep(delay)
            return process_batch_json_array(*args, **kwargs)
        except Exception as e:
            msg = str(e)
            if _is_quota_or_rate_error(msg):
                switched = rotator.rotate()
                if not switched:
                    delay = min(max(5.0, (delay or 2.0) * 1.8), 180.0)
                else:
                    delay = 1.0
                time.sleep(delay + random.uniform(0, 0.7))
                continue
            if _is_transient_server_error(msg):
                delay = min(max(4.0, (delay or 2.0) * 1.6), 180.0)
                time.sleep(delay + random.uniform(0, 1.2))
                continue
            raise
    return [], False

def safe_process_single_image(p: Path, max_retries=MAX_RETRIES) -> str:
    delay = 1.0
    for attempt in range(max_retries):
        try:
            return process_single_image(
                build_client(), MODEL, p, LANG,
                INLINE_LIMIT_MB, THINKING_BUDGET, MAX_OUTPUT_TOKENS, TEMPERATURE
            )
        except Exception as e:
            msg = str(e)
            if _is_quota_or_rate_error(msg):
                switched = rotator.rotate()
                if not switched:
                    delay = min(max(5.0, delay * 1.8), 180.0)
                else:
                    delay = 1.0
                time.sleep(delay + random.uniform(0, 0.7))
                continue
            if _is_transient_server_error(msg):
                delay = min(max(3.0, delay * 1.6), 180.0)
                time.sleep(delay + random.uniform(0, 1.0))
                continue
            raise
    return ""

# ---------- NORMALIZE NEWLINES ----------
def normalize_text_newlines(s: str) -> str:
    if not isinstance(s, str):
        return ""
    s = s.replace("\r\n", "\n").replace("\r", "\n")
    if NORMALIZE_TEXT_MODE == "keep":
        return s
    elif NORMALIZE_TEXT_MODE == "escape":
        return s.replace("\n", "\\n")
    elif NORMALIZE_TEXT_MODE == "one_line":
        return " ".join(s.splitlines())
    else:
        return s

# ---------- RUNNER ----------
ensure_dir(OCR_TXT_DIR)

done_set = load_done(CHECKPOINT)

images_all = list_images(INPUT_DIR)
images = [p for p in images_all if p.name not in done_set]
if MAX_IMAGES and MAX_IMAGES > 0:
    images = images[:MAX_IMAGES]

print(f"▶️ Total images: {len(images_all)} | To process: {len(images)} | init batch={INIT_BATCH_SIZE}")

def write_result_record(jsonl_f, p: Path, txt: str):
    txt_norm = normalize_text_newlines(txt)
    (OCR_TXT_DIR/p.stem).with_suffix(".txt").write_text(txt_norm, encoding="utf-8")
    jsonl_f.write(json.dumps({"image": str(p), "text": txt_norm}, ensure_ascii=False) + "\n")

def run_adaptive(images: List[Path], init_bs: int):
    bs = max(1, int(init_bs))
    i = 0
    failed_names = []
    with open(OUT_JSONL, "a", encoding="utf-8") as f_jsonl:
        pbar = tqdm(total=len(images))
        while i < len(images):
            batch = images[i:i+bs]

            arr, ok = safe_process_batch_json_array(
                build_client(), MODEL, batch, LANG,
                INLINE_LIMIT_MB, THINKING_BUDGET, MAX_OUTPUT_TOKENS, TEMPERATURE,
                max_retries=MAX_RETRIES, throttle_sec=THROTTLE_SEC
            )

            if not ok and bs > 1:
                bs = max(1, bs // 2)
                print(f"⚠️ Incomplete batch → shrink batch size to {bs} and retry same index")
                time.sleep(3)
                continue

            if not ok and bs == 1:
                p = batch[0]
                try:
                    txt = safe_process_single_image(p, max_retries=MAX_RETRIES)
                except Exception:
                    txt = ""
                    failed_names.append(p.name)

                write_result_record(f_jsonl, p, txt)
                append_done(CHECKPOINT, [p.name])
                i += 1
                pbar.update(1)
                bs = min(init_bs, bs + 1)
                continue

            by_name = { r.get("image",""): r.get("text","") for r in arr }
            just_done = []
            for p in batch:
                txt = by_name.get(p.name, "")
                if not isinstance(txt, str):
                    txt = ""
                    failed_names.append(p.name)
                write_result_record(f_jsonl, p, txt)
                just_done.append(p.name)

            append_done(CHECKPOINT, just_done)
            i += len(batch)
            pbar.update(len(batch))
        pbar.close()
    return failed_names

failed = run_adaptive(images, INIT_BATCH_SIZE)

print(f"\n✅ Finished.\n- JSONL : {OUT_JSONL}\n- TXT   : {OCR_TXT_DIR}\n- Check : {CHECKPOINT}")
if failed:
    print(f"⚠️ {len(failed)} file(s) ended up with empty text (kept as ''). You can re-run those.")

# ---------- EXPORT CSV ----------
if EXPORT_CSV:
    import csv
    rows = []
    if OUT_JSONL.exists():
        with open(OUT_JSONL, "r", encoding="utf-8") as f:
            for ln in f:
                ln = ln.strip()
                if not ln: 
                    continue
                try:
                    obj = json.loads(ln)
                    rows.append(obj)
                except:
                    pass
    df = pd.DataFrame(rows)
    csv_path = OUT_JSONL.with_suffix(".csv")
    df.to_csv(csv_path,
              index=False,
              encoding="utf-8-sig",
              quoting=csv.QUOTE_ALL,
              lineterminator="\n")
    print("CSV saved to:", csv_path)


▶️ Total images: 275 | To process: 118 | init batch=5


  0%|          | 0/118 [00:00<?, ?it/s]

⚠️ Incomplete batch → shrink batch size to 2 and retry same index


100%|██████████| 118/118 [06:39<00:00,  3.38s/it]


✅ Finished.
- JSONL : C:\projecteve\OCR\data\yolo_crop\OCR_LLM\ocr_output_2025-11-04.jsonl
- TXT   : C:\projecteve\OCR\data\yolo_crop\OCR_LLM\ocr_txt_2025-11-04
- Check : C:\projecteve\OCR\data\yolo_crop\OCR_LLM\ocr_done_list_2025-11-04.txt
CSV saved to: C:\projecteve\OCR\data\yolo_crop\OCR_LLM\ocr_output_2025-11-04.csv


## LLM selection

In [ ]:
# -*- coding: utf-8 -*-
# ===========================================================
# Thai Full-Image Text Correction (Token-by-Token, Batch + OCR fallback)
# - เลือกคำต่อคำระหว่าง EasyOCR กับ LLM-OCR ด้วยภาพจริงเป็นหลัก
# - ถ้าไม่มีคำไหนจาก CSV ที่ตรงภาพ → OCR คำใหม่ (fallback)
# - Batch หลายตำแหน่งในครั้งเดียว เพื่อลดเวลา/ค่าใช้จ่าย
# - Output:
#   CSV   => (ขึ้นกับ ONLY_PRODUCT_NAME) image, product_name[, text_full, text_llm_only]
#   JSONL => เก็บรายละเอียดการตัดสินใจทั้งหมด (ต่อภาพ)
# - ทุกฟิลด์ข้อความถูกบีบให้เป็น "บรรทัดเดียว" เสมอ
# ===========================================================
# pip install -q google-genai pillow pandas tqdm
# (optional): pip install pythainlp regex rapidfuzz

from pathlib import Path
import os, re, json, time, random, tempfile, sys, difflib
from typing import List, Dict, Tuple, Any, Optional
import pandas as pd
from PIL import Image
from google import genai
from google.genai import types
from tqdm import tqdm

# ===================== CONFIG =====================
# --- API Keys ---
KEY_POOL = [
    "Press Key Here",
    "Press Key Here",
]
assert len([k for k in KEY_POOL if k.strip()]) > 0, "กรุณาใส่ API key อย่างน้อย 1 ค่า"

# --- Inputs (ปรับตามไฟล์ของคุณ) ---
CSV_FILES = [
    ("Easy", r"C:\projecteve\OCR\data\yolo_crop\output_detail_only2025-11-04\ocr_output_train_CJ.csv"),
    ("LLM",  r"C:\projecteve\OCR\data\yolo_crop\OCR_LLM\ocr_output_2025-11-04.csv"),
]
IMAGE_ROOT = Path(r"C:\projecteve\OCR\data\yolo_crop\output_detail_only2025-11-04")

# --- Outputs ---
OUT_JSONL = Path(r"C:\projecteve\OCR\data\yolo_crop\OCR_LLM\LLM_data-correction\fulltext_pick_2025-11-04.jsonl")
OUT_CSV   = Path(r"C:\projecteve\OCR\data\yolo_crop\OCR_LLM\LLM_data-correction\fulltext_pick_2025-11-04.csv")

# --- Export mode ---
ONLY_PRODUCT_NAME = True   # True = CSV จะมีแค่ image, product_name | False = บวก text_full, text_llm_only

# --- Model config ---
MODEL              = "gemini-2.5-flash"
TEMPERATURE        = 0.05
TOP_P              = 0.2
TOP_K              = 1
THINKING_BUDGET    = 0
INLINE_LIMIT_MB    = 2.0
MAX_OUTPUT_TOKENS  = 192
THROTTLE_SEC       = 0.4
MAX_RETRIES        = 3

RUN_ID             = time.strftime("%Y%m%d-%H%M%S")
MAX_SIDE           = 1280
JPEG_QUALITY       = 75

MAX_CANDIDATES_PER_POS = 3
USE_BATCH          = True
BATCH_POS          = 24

# ===================== Optional libs =====================
try:
    from pythainlp import word_tokenize as th_word_tokenize
except Exception:
    th_word_tokenize = None

try:
    import regex as re_grx
except Exception:
    re_grx = None

try:
    from rapidfuzz.distance import Levenshtein
except Exception:
    Levenshtein = None

# ===================== Utils =====================
def log(*args): print(*args, file=sys.stderr)

CONTROL_RE = re.compile(r'[\x00-\x08\x0B\x0C\x0E-\x1F]')
THAI_UNIT_TOKENS = ["มล.","ลิตร","กรัม","ก.","กก.","ขวด","ซอง","กล่อง","แพ็ค","แพค","แพ็คคู่","ชิ้น","แผง","ชุด","แบบ","ไซส์","ไซซ์","ถุง","ขนัด"]

def ensure_dir(p: Path):
    p.parent.mkdir(parents=True, exist_ok=True)

def clear_outputs():
    for p in [OUT_JSONL, OUT_CSV]:
        try:
            if p.exists(): p.unlink()
        except: pass

def to_one_line(s: str) -> str:
    if s is None:
        return ""
    return re.sub(r"\s+", " ", str(s)).strip()

def try_read_csv(path: Path) -> Optional[pd.DataFrame]:
    if not path or not Path(path).exists():
        log(f"⚠️ CSV not found: {path}")
        return None
    for enc in ["utf-8-sig", "utf-8", "cp874", "cp1252", "latin1"]:
        try:
            return pd.read_csv(path, encoding=enc)
        except Exception:
            continue
    try:
        return pd.read_csv(path, engine="python")
    except Exception as e:
        log(f"❌ cannot read CSV: {path} -> {e}")
        return None

# ---------- Client with key rotation ----------
class KeyRotator:
    def __init__(self, keys: List[str]):
        ks = [k.strip() for k in keys if k and k.strip()]
        if not ks: raise ValueError("KEY_POOL is empty")
        self.keys = ks
        self.idx = 0
        self._client = None
        self._init_client()

    def _init_client(self):
        os.environ["GEMINI_API_KEY"] = self.keys[self.idx]
        self._client = genai.Client(api_key=self.keys[self.idx])

    def client(self) -> "genai.Client":
        return self._client

    def rotate(self) -> bool:
        if len(self.keys) == 1:
            time.sleep(1.0)
            return False
        self.idx = (self.idx + 1) % len(self.keys)
        log(f"🔑 Switch API key → {self.idx+1}/{len(self.keys)}")
        self._init_client()
        return True

rotator = KeyRotator(KEY_POOL)
def build_client() -> "genai.Client":
    return rotator.client()

# ---------- Image helper ----------
def _infer_mime(suffix: str) -> str:
    s = suffix.lower().lstrip(".")
    return {"png":"image/png","jpg":"image/jpeg","jpeg":"image/jpeg","webp":"image/webp"}.get(s,"application/octet-stream")

def to_bytes(p: Path) -> bytes:
    with open(p, "rb") as f: return f.read()

def prepare_image_for_upload(p: Path) -> Path:
    try:
        im = Image.open(p).convert("RGB")
        w, h = im.size
        scale = min(1.0, MAX_SIDE / max(w, h))
        if scale < 1.0:
            im = im.resize((max(1,int(w*scale)), max(1,int(h*scale))), Image.LANCZOS)
        tmp = Path(tempfile.gettempdir()) / f"llm_img_{p.stem}_{im.size[0]}x{im.size[1]}.jpg"
        im.save(tmp, format="JPEG", quality=JPEG_QUALITY, optimize=True)
        return tmp
    except Exception as e:
        log(f"⚠️ prepare_image_for_upload failed ({p}): {e}")
        return p

def make_image_part(p: Path) -> Any:
    client = build_client()
    try:
        size_mb = p.stat().st_size / (1024*1024)
    except:
        size_mb = INLINE_LIMIT_MB + 1.0
    if size_mb <= INLINE_LIMIT_MB:
        try:
            return types.Part.from_bytes(data=to_bytes(p), mime_type=_infer_mime(p.suffix))
        except Exception as e:
            log(f"⚠️ inline image failed, will upload: {e}")
    up = prepare_image_for_upload(p)
    try:
        return client.files.upload(file=str(up))
    except Exception as e:
        log(f"❌ image upload failed ({p}): {e}")
        try:
            return types.Part.from_bytes(data=to_bytes(up), mime_type=_infer_mime(up.suffix))
        except:
            return types.Part.from_bytes(data=b"", mime_type="application/octet-stream")

IMAGE_PART_CACHE: Dict[str, Any] = {}
def get_image_part_for(p: Path):
    key = str(p.resolve())
    if key not in IMAGE_PART_CACHE:
        IMAGE_PART_CACHE[key] = make_image_part(p)
    return IMAGE_PART_CACHE[key]

# ---------- Image index / resolve ----------
def index_images(root: Path, exts=(".png",".jpg",".jpeg",".webp")) -> Dict[str, List[Path]]:
    idx: Dict[str, List[Path]] = {}
    if root and root.exists():
        for p in root.rglob("*"):
            if p.suffix.lower() in exts and p.is_file():
                idx.setdefault(p.name.lower(), []).append(p.resolve())
    return idx

IMAGE_INDEX = {}
def resolve_image_path(img_str: str) -> Optional[Path]:
    s = str(img_str).strip().strip('"').strip("'")
    if not s: return None
    p = Path(s)
    if p.is_absolute() and p.exists(): return p.resolve()
    if (Path.cwd() / p).exists(): return (Path.cwd() / p).resolve()
    if IMAGE_ROOT and (IMAGE_ROOT / p).exists(): return (IMAGE_ROOT / p).resolve()
    base = Path(s).name.lower()
    cands = IMAGE_INDEX.get(base, [])
    if cands:
        try:
            root_s = str(IMAGE_ROOT.resolve()).lower()
            prefer = [c for c in cands if str(c).lower().startswith(root_s)]
            return prefer[0] if prefer else cands[0]
        except:
            return cands[0]
    return None

# ---------- Text normalize / tokenize ----------
def clean_for_all_text(s: str) -> str:
    if not s: return ""
    t = str(s)
    t = re.sub(r"^\s*\d+\s*[\.\)\-:]\s*", "", t.strip())
    t = CONTROL_RE.sub("", t)
    t = re.sub(r"\s+", " ", t).strip()
    return t

def tokenize_all_text(text: str) -> List[str]:
    t = clean_for_all_text(text or "")
    if not t: return []
    if th_word_tokenize:
        try:
            toks = [w.strip() for w in th_word_tokenize(t, engine="newmm") if w.strip()]
        except Exception:
            toks = [w for w in re.split(r"[ \t\n\r\f\v/|]+", t) if w]
    else:
        toks = [w for w in re.split(r"[ \t\n\r\f\v/|]+", t) if w]
    return toks

def join_tokens_all(tokens: List[str]) -> str:
    s = " ".join([t for t in tokens if t]).strip()
    s = re.sub(r"\s+", " ", s).strip()
    return s

# ---------- CSV loader & collect text ----------
def load_rows(csv_path: Path) -> Dict[str, dict]:
    df = try_read_csv(csv_path)
    if df is None or df.empty:
        return {}
    image_col = None
    for c in df.columns:
        if str(c).lower() in ["image","img","image_path","filename","file","path"]:
            image_col = c; break
    if image_col is None:
        image_col = df.columns[0]
        log(f"⚠️ 'image' column not found in {csv_path.name}. Using first column: {image_col}")
    out = {}
    for _, r in df.iterrows():
        img = str(r.get(image_col, "") if pd.notna(r.get(image_col, "")) else "").strip()
        if not img: continue
        row = {}
        for c in df.columns:
            v = r.get(c)
            row[c] = v
            row[str(c).lower()] = v
        out[img] = row
    return out

POSSIBLE_TEXT_COLS = ["text","product_name","name","title","desc","description","ocr_text","full_text"]
POSSIBLE_CODE_COLS = ["product_no","product_code","code","sku","item_code","รหัส","รหัสสินค้า"]
POSSIBLE_SIZE_COLS = ["size","unit","size_text","pack","pack_size","quantity","volume","weight"]
POSSIBLE_PRICE_COLS= ["price","final_price","discounted_price","sale_price","current_price","net_price","ราคา"]
POSSIBLE_WAS_COLS  = ["normal_price","original_price","was_price","list_price","ราคาเดิม"]

def collect_alltext_by_source(rows_for_img: Dict[str, dict]) -> Dict[str, str]:
    by_src: Dict[str, str] = {}
    all_cols = set(POSSIBLE_TEXT_COLS + POSSIBLE_CODE_COLS + POSSIBLE_SIZE_COLS + POSSIBLE_PRICE_COLS + POSSIBLE_WAS_COLS)
    for lab, row in rows_for_img.items():
        parts = []
        for cols in [POSSIBLE_TEXT_COLS, POSSIBLE_CODE_COLS, POSSIBLE_SIZE_COLS, POSSIBLE_PRICE_COLS, POSSIBLE_WAS_COLS]:
            for c in cols:
                if c in row and pd.notna(row[c]) and str(row[c]).strip():
                    parts.append(str(row[c]).strip())
        for c, v in row.items():
            if c in all_cols: continue
            if pd.notna(v) and isinstance(v, (str, int, float)):
                sv = str(v).strip()
                if sv: parts.append(sv)
        by_src[lab] = " ".join(parts).strip()
    return by_src

# ---------- Alignment ----------
def multi_align_positions(tokens_by_src: Dict[str, List[str]]) -> List[List[Tuple[str, str]]]:
    if not tokens_by_src:
        return []
    pivot_label = max(tokens_by_src.keys(), key=lambda k: len(tokens_by_src[k]))
    pivot = tokens_by_src[pivot_label]
    positions: List[List[Tuple[str, str]]] = [[(pivot_label, w)] for w in pivot]
    if not positions:
        for lab, toks in tokens_by_src.items():
            if toks:
                positions = [[(lab, w)] for w in toks]
                pivot_label = lab
                pivot = toks
                break
    for lab, toks in tokens_by_src.items():
        if lab == pivot_label: continue
        sm = difflib.SequenceMatcher(a=pivot, b=toks, autojunk=False)
        pos_cursor = 0
        for tag, i1, i2, j1, j2 in sm.get_opcodes():
            if tag == "equal":
                for k in range(i2 - i1):
                    positions[i1 + k].append((lab, toks[j1 + k]))
                pos_cursor = i2
            elif tag == "replace":
                pair = min(i2 - i1, j2 - j1)
                for k in range(pair):
                    if i1 + k < len(positions):
                        positions[i1 + k].append((lab, toks[j1 + k]))
                extra_b = (j2 - j1) - pair
                for k in range(extra_b):
                    insert_at = min(i2 + k, len(positions))
                    positions.insert(insert_at, [(lab, toks[j1 + pair + k])])
                pos_cursor = min(i2 + max(0, extra_b), len(positions))
            elif tag == "insert":
                for k in range(j1, j2):
                    insert_at = min(pos_cursor, len(positions))
                    positions.insert(insert_at, [(lab, toks[k])])
                    pos_cursor = min(pos_cursor + 1, len(positions))
            elif tag == "delete":
                pos_cursor = i2
    return positions

def is_trivial(tok: str) -> bool:
    if not tok: return False
    s = tok.strip()
    if re.fullmatch(r"[\d.,:/\-+()%฿xX]+", s): return True
    if any(u in s for u in THAI_UNIT_TOKENS): return True
    return False

def trim_candidates(cands: List[Tuple[str,str]]) -> List[Tuple[str,str]]:
    seen = set(); out = []
    for lab, w in cands:
        w = (w or "").strip()
        if not w: continue
        key = (lab, w)
        if key in seen: continue
        seen.add(key)
        out.append((lab, w))
        if len(out) >= MAX_CANDIDATES_PER_POS: break
    values = {w for _, w in out}
    if len(values) == 1 and out:
        return [out[0]]
    return out

# ===================== Product Name helpers =====================
NAME_COLS_PRIMARY   = ["product_name", "name", "title"]
PRICE_KEYWORDS_TH   = ["ปกติ", "พิเศษ", "ราคา", "บาท", "ลด", "โปร", "แถม", "ซื้อ", "ชิ้น", "แพ็ค", "แพค"]
UNIT_PAT = r"(มล\.|ลิตร|กรัม|ก\.|กก\.|ขวด|ซอง|กล่อง|แพ็ค|แพค|ชิ้น|แผง|ชุด|แบบ|ไซส์|ไซซ์)"
PRICE_PAT = r"(฿\s*\d[\d.,]*|\d[\d.,]*\s*(บาท|฿)|\d+%|\d+\s*-\.)"

def _penalize_noise(s: str) -> int:
    p = 0
    if re.search(r"\d", str(s)): p += 2
    if any(k in str(s) for k in PRICE_KEYWORDS_TH): p += 2
    if re.search(UNIT_PAT, str(s)): p += 1
    if len(str(s).split()) <= 1: p += 1
    return p

def _name_score(s: str) -> float:
    core = re.sub(r"[^A-Za-zก-๙\s\-&()/\.]", "", str(s)).strip()
    return max(0, len(core)) - _penalize_noise(str(s))

def _clean_from_fulltext(text: str) -> str:
    if not text: return ""
    t = str(text)
    for kw in PRICE_KEYWORDS_TH:
        if kw in t:
            t = t.split(kw)[0]
    t = re.sub(UNIT_PAT, "", t)
    t = re.sub(PRICE_PAT, "", t)
    t = re.sub(r"\b\d[\d.,]*\b", "", t)
    t = re.sub(r"[^A-Za-zก-๙\s\-&()/\.]", " ", t)
    t = re.sub(r"\s+", " ", t).strip()
    toks = t.split()
    if len(toks) > 10:
        t = " ".join(toks[:10])
    return t

def pick_product_name(rows_for_img: dict, text_full: str) -> str:
    cands = []
    for row in rows_for_img.values():
        for c in NAME_COLS_PRIMARY + POSSIBLE_TEXT_COLS:
            v = row.get(c) or row.get(str(c).lower())
            if v and isinstance(v, (str, int, float)):
                sv = str(v).strip()
                if sv:
                    cands.append(sv)
    scored = sorted(cands, key=lambda s: _name_score(str(s)), reverse=True)
    if scored and _name_score(scored[0]) >= 3:
        return to_one_line(scored[0])
    h = _clean_from_fulltext(text_full)
    return to_one_line(h)

# ===================== Prompts =====================
def prompt_field_bootstrap(run_id: str, image_path: str) -> str:
    return f"""RUN_ID={run_id}
THAI FIELD OCR BOOTSTRAP — อ่าน 'ข้อความทั้งหมดที่เห็นในภาพ' โดยไม่อ้างอิง CSV:
- image={image_path}
ข้อกำหนด:
1) อ่านเฉพาะข้อความที่ปรากฏจริง (ตามลำดับการอ่านตามภาพ)
2) ห้ามเดา/แต่งเติม ถ้าไม่เห็นให้เว้นว่าง
3) ตอบ JSON เท่านั้น:
{{
  "text": "<สตริงข้อความทั้งหมดในภาพ>"
}}"""

def prompt_token_pick_batch(run_id: str, image_path: str,
                            batch_positions: List[dict],
                            context_texts: Dict[str, str]) -> str:
    ctx_lines  = [f"- {lab}_full: {to_one_line(context_texts.get(lab,''))}" for lab in sorted(context_texts.keys())]
    trimmed_items = []
    for it in batch_positions:
        c = it.get("candidates", [])
        trimmed_items.append({"idx": it.get("idx", -1),
                              "candidates": [[lab, w] for lab, w in c]})
    return (
f"""RUN_ID={run_id}
THAI TOKEN PICK (BATCH) — เลือกโทเค็นต่อโทเค็นจาก CSV ด้วยภาพจริง:
- image={image_path}

กติกาต่อ item:
1) ถ้าโทเค็นจาก candidates ตรงกับข้อความบนภาพ → เลือก (picked_label, picked_token)
2) ถ้าไม่มีโทเค็นใดใน candidates ที่ตรงกับภาพ → ตั้ง ocr_fallback=true และถอดจากภาพเป็น new_token
3) ห้ามแต่งคำใหม่/ผสมคำจากหลายแหล่งเอง

context (สำหรับอ้างอิงการอ่านภาพเท่านั้น):
""" + "\n".join(ctx_lines) + """

ตอบเป็น JSON array เท่านั้น:
[
  {
    "idx": <int>,
    "picked_label": "<label หรือ \"\">",
    "picked_token": "<โทเค็นจาก CSV หรือ \"\">",
    "confidence": <0.0-1.0>,
    "ocr_fallback": <true|false>,
    "new_token": "<ถ้า OCR-fallback; มิฉะนั้น \"\">"
  },
  ...
]

items:
""" + json.dumps(trimmed_items, ensure_ascii=False)
)

# ===================== LLM calls =====================
def _safe_load_json(text: str):
    if not text:
        return None
    s = CONTROL_RE.sub("", text).strip()
    if "```" in s:
        parts = [p for p in s.split("```") if p.strip()]
        s = max(parts, key=len)
    s = s.strip()
    try:
        return json.loads(s)
    except Exception:
        try:
            beg_obj, beg_arr = s.find("{"), s.find("[")
            if beg_obj == -1 and beg_arr == -1:
                return None
            beg = len(s) if beg_obj == -1 else beg_obj
            if beg_arr != -1: beg = min(beg, beg_arr)
            tail = s.rfind("}") if beg == beg_obj else s.rfind("]")
            if tail != -1 and tail > beg:
                return json.loads(s[beg:tail+1])
        except Exception:
            return None
    return None

def call_field_bootstrap(img_path: str, image_part: Any) -> str:
    client = build_client()
    parts = [prompt_field_bootstrap(RUN_ID, img_path), image_part]
    cfg = types.GenerateContentConfig(
        thinking_config=types.ThinkingConfig(thinking_budget=THINKING_BUDGET),
        max_output_tokens=MAX_OUTPUT_TOKENS,
        response_mime_type="application/json",
        temperature=TEMPERATURE, top_p=TOP_P, top_k=TOP_K,
    )
    delay = THROTTLE_SEC
    for attempt in range(MAX_RETRIES):
        try:
            if attempt == 0 and delay > 0: time.sleep(delay)
            resp = client.models.generate_content(model=MODEL, contents=parts, config=cfg)
            text = (getattr(resp, "text", None) or "").strip()
            obj = _safe_load_json(text)
            if isinstance(obj, dict):
                return to_one_line(str(obj.get("text","") or ""))
            return ""
        except Exception as e:
            m = str(e).lower()
            transient = any(k in m for k in [
                "remoteprotocolerror","server disconnected","disconnect","connection reset","connection aborted",
                "broken pipe","eof","timeout","temporarily unavailable","protocol error",
                "503","unavailable","internal","deadline","upstream"
            ])
            rate = any(k in m for k in ["429","quota","too many requests","exceeded","resourceexhausted","rate"])
            if transient:
                delay = min(max(3.0, (delay or 1.5)*1.6), 120.0)
                time.sleep(delay + random.uniform(0,0.7)); rotator.rotate(); continue
            if rate:
                if not rotator.rotate():
                    delay = min(max(4.0, (delay or 1.5)*1.8), 120.0)
                else:
                    delay = 0.8
                time.sleep(delay + random.uniform(0,0.5)); continue
            log(f"❌ field_bootstrap error: {e}")
            return ""
    return ""

def call_token_picker_batch(img_path: str,
                            items: List[dict],
                            context_texts: Dict[str, str],
                            image_part: Any) -> List[dict]:
    if not items: return []
    client = build_client()
    prompt = prompt_token_pick_batch(RUN_ID, img_path, items, context_texts)
    parts = [prompt, image_part]
    cfg = types.GenerateContentConfig(
        thinking_config=types.ThinkingConfig(thinking_budget=THINKING_BUDGET),
        max_output_tokens=max(256, min(2048, 80 * len(items))),
        response_mime_type="application/json",
        temperature=TEMPERATURE, top_p=TOP_P, top_k=TOP_K,
    )
    delay = THROTTLE_SEC
    for attempt in range(MAX_RETRIES):
        try:
            if attempt == 0 and delay > 0: time.sleep(delay)
            resp = client.models.generate_content(model=MODEL, contents=parts, config=cfg)
            text = (getattr(resp, "text", None) or "").strip()
            arr = _safe_load_json(text)
            out = []
            if isinstance(arr, dict):
                arr = arr.get("results", [])
            if isinstance(arr, list):
                for obj in arr:
                    try:
                        out.append({
                            "idx"          : int(obj.get("idx", -1)),
                            "picked_label" : to_one_line(str(obj.get("picked_label","") or "")),
                            "picked_token" : to_one_line(str(obj.get("picked_token","") or "")),
                            "confidence"   : float(obj.get("confidence",0.0) or 0.0),
                            "ocr_fallback" : bool(obj.get("ocr_fallback", False)),
                            "new_token"    : to_one_line(str(obj.get("new_token","") or "")),
                        })
                    except Exception:
                        pass
            return out
        except Exception as e:
            m = str(e).lower()
            transient = any(k in m for k in [
                "remoteprotocolerror","server disconnected","disconnect","connection reset","connection aborted",
                "broken pipe","eof","timeout","temporarily unavailable","protocol error",
                "503","unavailable","internal","deadline","upstream"
            ])
            rate = any(k in m for k in ["429","quota","too many requests","exceeded","resourceexhausted","rate"])
            if transient:
                delay = min(max(3.0, (delay or 1.5)*1.6), 120.0)
                time.sleep(delay + random.uniform(0,0.7)); rotator.rotate(); continue
            if rate:
                if not rotator.rotate():
                    delay = min(max(4.0, (delay or 1.5)*1.8), 120.0)
                else:
                    delay = 0.8
                time.sleep(delay + random.uniform(0,0.5)); continue
            log(f"❌ token_picker_batch error: {e}")
            return [{"idx": it.get("idx",-1), "picked_label":"", "picked_token":"", "confidence":0.0, "ocr_fallback":False, "new_token":"", "notes": f"err:{e!r}"} for it in items]
    return [{"idx": it.get("idx",-1), "picked_label":"", "picked_token":"", "confidence":0.0, "ocr_fallback":False, "new_token":"", "notes":"retry_exhausted"} for it in items]

# ===================== Main =====================
def main():
    global IMAGE_INDEX
    if not IMAGE_ROOT.exists():
        log(f"⚠️ IMAGE_ROOT not found: {IMAGE_ROOT}")
    IMAGE_INDEX = index_images(IMAGE_ROOT)

    ensure_dir(OUT_JSONL); ensure_dir(OUT_CSV)
    clear_outputs()
    open(OUT_JSONL, "w", encoding="utf-8").close()

    # โหลด CSV → dict image_key -> row
    rows_by_label: Dict[str, Dict[str, dict]] = {}
    for lab, pth in CSV_FILES:
        rows_by_label[lab] = load_rows(Path(pth))
        log(f"📄 {lab} CSV keys (raw): {len(rows_by_label[lab])}")

    # mapping: resolved image path -> set(keys ที่อ้างถึงรูปนั้น)
    resolved_to_keys: Dict[str, set] = {}
    for _, mp in rows_by_label.items():
        for k in mp.keys():
            rp = resolve_image_path(k) or resolve_image_path(Path(k).name)
            if rp is None: continue
            rps = str(rp)
            resolved_to_keys.setdefault(rps, set()).add(k)
            resolved_to_keys[rps].add(Path(k).name)

    all_resolved = sorted(resolved_to_keys.keys())
    log(f"🧮 Images (unique, resolved): {len(all_resolved)}")

    out_rows = []  # สำหรับ CSV

    for resolved_str in tqdm(all_resolved, desc="Token-by-Token pick (batch)"):
        img_path = Path(resolved_str)
        if not img_path.exists():
            log(f"[skip] image not found: {resolved_str}")
            continue

        # รวมแถวตาม key ที่ match รูปนี้
        possible_keys = resolved_to_keys[resolved_str]
        rows_for_img: Dict[str, dict] = {}
        for lab, mp in rows_by_label.items():
            found = None
            for k in possible_keys:
                if k in mp: found = mp[k]; break
            if not found:
                for k in possible_keys:
                    base = Path(k).name
                    if base in mp: found = mp[base]; break
            if not found and resolved_str in mp:
                found = mp[resolved_str]
            if found:
                rows_for_img[lab] = found

        if not rows_for_img:
            log(f"[skip] no rows for resolved image: {resolved_str}")
            continue

        # รวมข้อความทุกแหล่ง
        alltext_by_src = {lab: to_one_line(txt) for lab, txt in collect_alltext_by_source(rows_for_img).items()}
        tokens_by_src: Dict[str, List[str]] = {lab: tokenize_all_text(txt) for lab, txt in alltext_by_src.items()}

        image_part = get_image_part_for(img_path)

        # ถ้า CSV ว่างทุกแหล่ง → OCR ทั้งภาพ (bootstrap)
        if not any(tokens_by_src.values()):
            boot_txt = call_field_bootstrap(str(img_path), image_part)
            final_text = to_one_line(clean_for_all_text(boot_txt)) if boot_txt else ""
            product_name = to_one_line(pick_product_name(rows_for_img, final_text))
            with open(OUT_JSONL, "a", encoding="utf-8") as fout:
                fout.write(json.dumps({
                    "run_id":RUN_ID,"image":resolved_str,"image_key":Path(resolved_str).name,
                    "text_full":final_text,"text_llm_only":final_text,"product_name":product_name,
                    "decisions":[{"mode":"bootstrap"}]}, ensure_ascii=False) + "\n")
            out_rows.append({"image":resolved_str, "product_name": product_name,
                             "text_full": final_text, "text_llm_only": final_text})
            continue

        # จัดตำแหน่ง
        positions = multi_align_positions(tokens_by_src)
        total_pos = len(positions)
        if total_pos == 0:
            boot_txt = call_field_bootstrap(str(img_path), image_part)
            final_text = to_one_line(clean_for_all_text(boot_txt)) if boot_txt else ""
            product_name = to_one_line(pick_product_name(rows_for_img, final_text))
            with open(OUT_JSONL, "a", encoding="utf-8") as fout:
                fout.write(json.dumps({
                    "run_id":RUN_ID,"image":resolved_str,"image_key":Path(resolved_str).name,
                    "text_full":final_text,"text_llm_only":final_text,"product_name":product_name,
                    "decisions":[{"mode":"bootstrap_empty_positions"}]}, ensure_ascii=False) + "\n")
            out_rows.append({"image":resolved_str, "product_name":product_name,
                             "text_full": final_text, "text_llm_only": final_text})
            continue

        final_tokens = [""] * total_pos
        llm_only_tokens = [""] * total_pos
        decisions: List[dict] = []

        need_llm_items = []
        for pos_idx, cands in enumerate(positions):
            cands = trim_candidates(cands)
            cand_vals = [w for _, w in cands if w]

            if not cand_vals:
                need_llm_items.append({"idx": pos_idx, "candidates": []})
                continue

            if len(set(cand_vals)) == 1:
                tok = cand_vals[0]
                final_tokens[pos_idx] = tok
                decisions.append({"pos":pos_idx,"total":total_pos,"mode":"auto_equal","token":tok,"candidates":cands})
                continue

            if any(is_trivial(w) for w in cand_vals) and len(set(cand_vals)) <= 2:
                tok = max(cand_vals, key=len)
                final_tokens[pos_idx] = tok
                decisions.append({"pos":pos_idx,"total":total_pos,"mode":"auto_trivial","token":tok,"candidates":cands})
                continue

            need_llm_items.append({"idx": pos_idx, "candidates": cands})

        force_round = []
        if need_llm_items:
            for i in range(0, len(need_llm_items), BATCH_POS if USE_BATCH else 1):
                chunk = need_llm_items[i:i+(BATCH_POS if USE_BATCH else 1)]
                items_for_prompt = [{"idx":it["idx"], "candidates":it["candidates"]} for it in chunk]
                res_list = call_token_picker_batch(str(img_path), items_for_prompt, alltext_by_src, image_part)
                res_by_idx = {r.get("idx"): r for r in res_list} if res_list else {}

                for it in chunk:
                    pos_idx = it["idx"]
                    cands   = it["candidates"]
                    res = res_by_idx.get(pos_idx, {}) if res_by_idx else {}
                    picked = to_one_line(res.get("picked_token") or "")
                    llm_conf = float(res.get("confidence", 0.0) or 0.0)
                    ocr_fb = bool(res.get("ocr_fallback", False))
                    new_tok = to_one_line(res.get("new_token") or "")

                    if ocr_fb:
                        tok = new_tok
                        final_tokens[pos_idx] = tok
                        llm_only_tokens[pos_idx] = tok
                        decisions.append({
                            "pos":pos_idx,"total":total_pos,"mode":"ocr_fallback",
                            "picked_label": "", "picked_token": "", "new_token": tok,
                            "llm_conf": llm_conf, "candidates": cands
                        })
                    else:
                        if picked and any(picked == v for _, v in cands):
                            tok = picked
                            final_tokens[pos_idx] = tok
                            llm_only_tokens[pos_idx] = tok
                            decisions.append({
                                "pos":pos_idx,"total":total_pos,"mode":"llm_pick",
                                "picked_label": to_one_line(res.get("picked_label","")),
                                "picked_token": tok, "new_token":"", "llm_conf": llm_conf, "candidates": cands
                            })
                        else:
                            force_round.append({"idx": pos_idx, "candidates": []})
                            decisions.append({
                                "pos":pos_idx,"total":total_pos,"mode":"force_ocr_pending",
                                "picked_label": to_one_line(res.get("picked_label","")),
                                "picked_token": picked, "new_token":"", "llm_conf": llm_conf, "candidates": cands
                            })
            if force_round:
                for i in range(0, len(force_round), BATCH_POS if USE_BATCH else 1):
                    chunk = force_round[i:i+(BATCH_POS if USE_BATCH else 1)]
                    res_list = call_token_picker_batch(str(img_path), chunk, alltext_by_src, image_part)
                    res_by_idx = {r.get("idx"): r for r in res_list} if res_list else {}
                    for it in chunk:
                        pos_idx = it["idx"]
                        res = res_by_idx.get(pos_idx, {}) if res_by_idx else {}
                        new_tok = to_one_line(res.get("new_token") or "")
                        if new_tok:
                            final_tokens[pos_idx] = new_tok
                            llm_only_tokens[pos_idx] = new_tok
                            decisions.append({
                                "pos": pos_idx, "total": total_pos, "mode": "force_ocr_done",
                                "picked_label":"", "picked_token":"", "new_token": new_tok,
                                "llm_conf": float(res.get("confidence",0.0) or 0.0), "candidates": []
                            })
                        else:
                            final_tokens[pos_idx] = ""
                            decisions.append({
                                "pos": pos_idx, "total": total_pos, "mode": "force_ocr_failed",
                                "picked_label":"", "picked_token":"", "new_token":"", "llm_conf": 0.0, "candidates": []
                            })

        # ------- สรุป text + product_name (บรรทัดเดียว) -------
        text_full = to_one_line(join_tokens_all(final_tokens))
        text_llm_only = to_one_line(join_tokens_all([t for t in llm_only_tokens if t]))
        product_name = to_one_line(pick_product_name(rows_for_img, text_full))

        out_json = {
            "run_id": RUN_ID,
            "image": resolved_str,
            "image_key": Path(resolved_str).name,
            "text_full": text_full,
            "text_llm_only": text_llm_only,
            "product_name": product_name,
            "decisions": decisions
        }
        with open(OUT_JSONL, "a", encoding="utf-8") as fout:
            fout.write(json.dumps(out_json, ensure_ascii=False) + "\n")

        out_rows.append({
            "image": resolved_str,
            "product_name": product_name,
            "text_full": text_full,
            "text_llm_only": text_llm_only
        })

    # ------- Save CSV -------
    ensure_dir(OUT_CSV)
    df = pd.DataFrame(out_rows)
    if ONLY_PRODUCT_NAME:
        df = df[["image", "product_name"]]
    else:
        df = df[["image","product_name","text_full","text_llm_only"]]
    df = df.applymap(to_one_line)
    df.to_csv(OUT_CSV, index=False, encoding="utf-8-sig", lineterminator="\n")

    print("✅ Done.")
    print("JSONL:", OUT_JSONL)
    print("CSV  :", OUT_CSV)

if __name__ == "__main__":
    main()


📄 Easy CSV keys (raw): 275
📄 LLM CSV keys (raw): 275
🧮 Images (unique, resolved): 275
Token-by-Token pick (batch):  34%|███▍      | 94/275 [08:53<08:58,  2.97s/it]🔑 Switch API key → 3/7
🔑 Switch API key → 4/7
🔑 Switch API key → 5/7
Token-by-Token pick (batch):  54%|█████▍    | 148/275 [14:57<11:46,  5.56s/it]🔑 Switch API key → 2/7
🔑 Switch API key → 3/7
🔑 Switch API key → 4/7
Token-by-Token pick (batch):  85%|████████▌ | 234/275 [23:59<03:25,  5.01s/it]🔑 Switch API key → 5/7
🔑 Switch API key → 6/7
🔑 Switch API key → 7/7
Token-by-Token pick (batch):  99%|█████████▉| 273/275 [28:10<00:08,  4.46s/it]🔑 Switch API key → 1/7
🔑 Switch API key → 2/7
Token-by-Token pick (batch): 100%|██████████| 275/275 [28:25<00:00,  6.20s/it]

✅ Done.
JSONL: C:\projecteve\OCR\data\yolo_crop\OCR_LLM\LLM_data-correction\fulltext_pick_2025-11-04.jsonl
CSV  : C:\projecteve\OCR\data\yolo_crop\OCR_LLM\LLM_data-correction\fulltext_pick_2025-11-04.csv



C:\Users\orawanya Eve\AppData\Local\Temp\ipykernel_20036\2360506313.py:810: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(to_one_line)


## 3. OCR.space

In [ ]:
# -*- coding: utf-8 -*-
"""
Robust OCR.Space runner for Thai (v2.2 anti-E101 + smart enlarge):
- ใช้ OCREngine=2 + language 'tha' (หรือ 'auto' ถ้าภาษาปน)
- จับ E101: Timed out waiting for results แล้ว fallback (ย่อภาพเพิ่ม/ลดคุณภาพ + เพิ่ม timeout)
- Retries + exponential backoff (+ jitter) บน timeout/5xx/429/E101
- ขยายภาพอัตโนมัติถ้าภาพเล็ก (LANCZOS) เพื่อให้ OCR อ่านชัดขึ้นอย่างพอดี
- เขียน CSV แบบ incremental และ resume ได้ (ข้ามไฟล์ที่สำเร็จแล้ว)
"""

import io, time, csv, sys, os, random
from pathlib import Path
from typing import Dict, Tuple
import requests
from requests.exceptions import ReadTimeout, ConnectTimeout, ConnectionError as ReqConnErr
import pandas as pd
from PIL import Image

# ================= CONFIG =================
API_URL  = "https://api.ocr.space/parse/image"
API_KEY  = "Key ocr.space".strip()  # <- key ของ ocr.space

# ภาษา/เอนจิน: ไทยต้องใช้ Engine 2 เสมอ
LANGUAGE = "tha"        # หรือ "auto" ถ้ารูปมีหลายภาษา
ENGINE   = 2            # ไทยใช้ได้ใน Engine 2 เท่านั้น

IMAGE_DIR  = Path(r"C:\projecteve\OCR\data\yolo_crop\output_detail_only2025-11-04")
OUTPUT_CSV = IMAGE_DIR / "ocr_results_2025-07-03.csv"

# Quota กัน rate-limit ฟรีแพลน (~180 req/ชั่วโมง)
SLEEP_BETWEEN_CALLS = 20       # วินาที พักระหว่างรูป
REQ_TIMEOUT_SEC     = 180      # ปกติ
REQ_TIMEOUT_FALLBACK_SEC = 240 # รอบ fallback (ยอมรอนานขึ้น)

# Retry policy
MAX_RETRIES   = 6
BACKOFF_BASE  = 2.0            # 2,4,8,16,...
RETRY_STATUS  = {408, 429, 500, 502, 503, 504}

# Image compress policy (ปกติ)
MAX_SIDE_PX        = 2400
JPEG_QUALITY       = 90
SKIP_COMPRESS_UNDER_MB = 2.5   # ถ้าไฟล์ต้นฉบับ < ค่านี้ (MB) จะส่งไฟล์เดิมเลย (ไม่บีบอัด)

# Image compress policy (fallback เมื่อเจอ E101/timeout)
FALLBACK_MAX_SIDE_PX  = 1600
FALLBACK_JPEG_QUALITY = 85

# Smart enlarge (ขยายเฉพาะภาพเล็ก)
ENLARGE_MIN_SIDE     = 800     # ถ้า short side < ค่านี้ → ขยาย
ENLARGE_MAX_UPSCALE  = 2.5     # ขยายสูงสุด 2.5×

# Debug overlay (เปิดชั่วคราวเพื่อดูตำแหน่งตัวอักษรจาก API)
IS_OVERLAY_REQUIRED = False    # ตั้ง True เฉพาะตอน debug

# ================= UTIL =================
def maybe_enlarge_image(im: Image.Image, min_side: int = ENLARGE_MIN_SIDE, max_upscale: float = ENLARGE_MAX_UPSCALE) -> Image.Image:
    """ขยายภาพถ้าเล็กกว่าที่กำหนด เพื่อให้ OCR อ่านชัดขึ้น (คุมไม่ให้ใหญ่เกิน)"""
    w, h = im.size
    short_side = min(w, h)
    if short_side < min_side:
        scale = min(max_upscale, float(min_side) / float(short_side))
        new_w, new_h = int(w * scale), int(h * scale)
        print(f"   🔍 upscale {w}x{h} → {new_w}x{new_h}")
        im = im.resize((new_w, new_h), resample=Image.LANCZOS)
    return im

def _prepare_rgb(im: Image.Image) -> Image.Image:
    """แปลงเป็น RGB + เคสภาพโปร่งใส"""
    if im.mode in ("RGBA", "LA"):
        bg = Image.new("RGB", im.size, (255, 255, 255))
        bg.paste(im, mask=im.split()[-1])
        return bg
    if im.mode != "RGB":
        return im.convert("RGB")
    return im

def _compress(path: Path, max_side: int, quality: int) -> bytes:
    """ย่อ/บีบอัดด้วย LANCZOS + enlarge สำหรับภาพเล็ก"""
    im = Image.open(path)
    im = _prepare_rgb(im)

    # ขยายถ้าภาพเล็กเกินไป
    im = maybe_enlarge_image(im, min_side=ENLARGE_MIN_SIDE, max_upscale=ENLARGE_MAX_UPSCALE)

    w, h = im.size
    s = min(1.0, max_side / max(w, h))
    if s < 1.0:
        im = im.resize((int(w * s), int(h * s)), resample=Image.LANCZOS)

    buf = io.BytesIO()
    im.save(buf, format="JPEG", quality=quality, optimize=True)
    return buf.getvalue()

def compress_primary(path: Path) -> bytes:
    return _compress(path, MAX_SIDE_PX, JPEG_QUALITY)

def compress_fallback(path: Path) -> bytes:
    return _compress(path, FALLBACK_MAX_SIDE_PX, FALLBACK_JPEG_QUALITY)

def read_file_bytes(path: Path) -> bytes:
    with open(path, "rb") as f:
        return f.read()

def maybe_prepare_image_bytes(path: Path) -> Tuple[bytes, str]:
    """
    ถ้าไฟล์ต้นฉบับเล็ก (< SKIP_COMPRESS_UNDER_MB) → ส่งไฟล์เดิม (รักษารายละเอียด)
    ถ้าใหญ่ → ย่อ/บีบอัดแบบ primary (พร้อม enlarge ถ้าจำเป็น)
    """
    try:
        size_mb = os.path.getsize(path) / (1024 * 1024.0)
    except:
        size_mb = SKIP_COMPRESS_UNDER_MB + 1  # กันพลาด บีบอัดไว้ก่อน
    if size_mb <= SKIP_COMPRESS_UNDER_MB:
        # แม้ส่งไฟล์เดิม เราก็อยาก enlarge ถ้าเล็กมาก → ต้องเข้าทาง compress
        # แต่เพื่อเรียบง่าย กติกาคือ ส่งไฟล์เดิมไปก่อน (รักษารายละเอียดสุด)
        # ถ้าไม่ผ่าน/E101 จะมาเข้า fallback ซึ่งจะเข้าทาง compress_fallback ที่ enlarge ให้
        return read_file_bytes(path), "raw"
    return compress_primary(path), "compressed"

def should_retry_api_error(body: dict) -> Tuple[bool, str]:
    """ตีความ error ฝั่ง API ที่ส่งกลับมากับ status 200 (เช่น E101)"""
    if not isinstance(body, dict):
        return False, ""
    if body.get("IsErroredOnProcessing"):
        msg = body.get("ErrorMessage") or body.get("ErrorDetails") or ""
        if isinstance(msg, list):
            msg = "; ".join([str(x) for x in msg if x])
        msg_low = str(msg).lower()
        if "e101" in msg_low or "timed out waiting for results" in msg_low or "timeout" in msg_low:
            return True, f"E101_or_timeout: {msg}"
        if any(k in msg_low for k in ["busy", "overloaded", "temporar", "please try again"]):
            return True, f"temporary_api_error: {msg}"
    return False, ""

def safe_post_with_retry(session: requests.Session, files: Dict, data: Dict, timeout_sec: int) -> Tuple[dict, str]:
    """
    ส่ง request พร้อม retry/backoff (+ jitter) เมื่อเจอ timeout/connection error/สถานะที่ควรลองใหม่
    """
    for attempt in range(1, MAX_RETRIES + 1):
        try:
            r = session.post(API_URL, files=files, data=data, timeout=timeout_sec)
            if r.status_code == 403:
                return {}, "HTTP 403 (forbidden/rate limit). Please wait and retry."
            if r.status_code != 200:
                if r.status_code in RETRY_STATUS:
                    raise ReadTimeout(f"Temporary status {r.status_code}: {r.text[:200]}")
                return {}, f"HTTP {r.status_code}: {r.text[:200]}"

            body = r.json()
            need_retry, reason = should_retry_api_error(body)
            if need_retry:
                if attempt == MAX_RETRIES:
                    return {}, f"{reason} after {MAX_RETRIES} retries"
                sleep_s = (BACKOFF_BASE ** (attempt - 1)) + random.uniform(0.0, 0.5)
                print(f"   ↪️ API body error -> retry {attempt}/{MAX_RETRIES-1} in {sleep_s:.1f}s ({reason})")
                time.sleep(sleep_s)
                continue

            return body, ""  # OK
        except (ReadTimeout, ConnectTimeout, ReqConnErr) as e:
            if attempt == MAX_RETRIES:
                return {}, f"network_or_timeout_after_{MAX_RETRIES}_retries: {repr(e)}"
            sleep_s = (BACKOFF_BASE ** (attempt - 1)) + random.uniform(0.0, 0.5)
            print(f"   ↪️ network/timeout -> retry {attempt}/{MAX_RETRIES-1} in {sleep_s:.1f}s due to: {repr(e)}")
            time.sleep(sleep_s)
        except Exception as e:
            return {}, f"unexpected_error: {repr(e)}"
    return {}, "unreachable_state"

def ocr_one(session: requests.Session, img_path: Path) -> Tuple[str, str]:
    """
    กลยุทธ์ 2 ช่วง:
      1) ส่งไฟล์ raw/primary-compress ด้วย timeout ปกติ
      2) ถ้า body แจ้ง E101/timeout หรือเครือข่าย timeout -> ลอง 'fallback' (enlarge + ย่อภาพเพิ่ม/ลดคุณภาพ + เพิ่ม timeout)
    """
    # ----- รอบแรก (ปกติ)
    img_bytes, mode = maybe_prepare_image_bytes(img_path)
    files = {"file": ("image.jpg", img_bytes, "image/jpeg")}
    data = {
        "apikey": API_KEY,
        "language": LANGUAGE,       # 'tha' หรือ 'auto'
        "OCREngine": ENGINE,        # ✅ พารามิเตอร์ถูกต้อง
        "isOverlayRequired": IS_OVERLAY_REQUIRED,
        "scale": True,
        "detectOrientation": True,
    }

    body, err = safe_post_with_retry(session, files, data, timeout_sec=REQ_TIMEOUT_SEC)
    if err:
        if any(k in err.lower() for k in ["timeout", "timed out", "e101", "temporary"]):
            print("   ↪️ fallback: shrink/enlarge & extend timeout")
        else:
            return "", f"{err} (mode={mode})"
    else:
        if body.get("IsErroredOnProcessing"):
            msg = body.get("ErrorMessage") or body.get("ErrorDetails") or "unknown_api_error"
            if isinstance(msg, list):
                msg = "; ".join([str(e) for e in msg if e])
            msg_low = str(msg).lower()
            if ("e101" in msg_low) or ("timed out waiting for results" in msg_low) or ("timeout" in msg_low):
                print("   ↪️ fallback: E101 in body -> shrink/enlarge & retry")
            else:
                return "", f"{msg} (mode={mode})"
        else:
            pr = body.get("ParsedResults") or []
            text = (pr[0].get("ParsedText") or "").strip() if pr else ""
            return text, ""  # success

    # ----- รอบ fallback (enlarge + ย่อภาพเพิ่ม/ลดคุณภาพ + เพิ่ม timeout)
    try:
        fb_bytes = compress_fallback(img_path)  # ภายในมี enlarge ให้ด้วย
    except Exception as e:
        return "", f"fallback_compress_failed: {repr(e)} (mode={mode})"

    files_fb = {"file": ("image.jpg", fb_bytes, "image/jpeg")}
    body2, err2 = safe_post_with_retry(session, files_fb, data, timeout_sec=REQ_TIMEOUT_FALLBACK_SEC)
    if err2:
        return "", f"{err2} (mode=fallback)"
    if body2.get("IsErroredOnProcessing"):
        msg2 = body2.get("ErrorMessage") or body2.get("ErrorDetails") or "unknown_api_error"
        if isinstance(msg2, list):
            msg2 = "; ".join([str(e) for e in msg2 if e])
        return "", f"{msg2} (mode=fallback)"

    pr2 = body2.get("ParsedResults") or []
    text2 = (pr2[0].get("ParsedText") or "").strip() if pr2 else ""
    return text2, ""  # success

def load_done_map(csv_path: Path) -> Dict[str, Dict[str, str]]:
    """อ่าน CSV เดิม (ถ้ามี) เพื่อ resume: ข้ามไฟล์ที่ text≠'' และ error=='' """
    if not csv_path.exists():
        return {}
    try:
        df = pd.read_csv(csv_path)
    except Exception as e:
        print(f"⚠️ Failed to read existing CSV ({csv_path}): {repr(e)}")
        return {}
    done = {}
    for _, row in df.iterrows():
        fname = str(row.get("filename"))
        if fname and fname.lower() != "filename":  # กันกรณีหัวตาราง
            done[fname] = {
                "text": str(row.get("text") if not pd.isna(row.get("text")) else ""),
                "error": str(row.get("error") if not pd.isna(row.get("error")) else "")
            }
    return done

def init_csv(csv_path: Path):
    csv_path.parent.mkdir(parents=True, exist_ok=True)
    with open(csv_path, "w", encoding="utf-8-sig", newline="") as f:
        csv.DictWriter(f, fieldnames=["filename", "text", "error"]).writeheader()

def append_row(csv_path: Path, row: Dict[str, str]):
    csv_path.parent.mkdir(parents=True, exist_ok=True)
    with open(csv_path, "a", encoding="utf-8-sig", newline="") as f:
        csv.DictWriter(f, fieldnames=["filename", "text", "error"]).writerow(row)

# ================= MAIN =================
if __name__ == "__main__":
    images = sorted([*IMAGE_DIR.glob("*.jpg"), *IMAGE_DIR.glob("*.jpeg"), *IMAGE_DIR.glob("*.png")])
    if not images:
        print(f"⚠️ No images found in: {IMAGE_DIR}")
        sys.exit(0)

    # เตรียม CSV (ถ้ายังไม่มี)
    if not OUTPUT_CSV.exists():
        init_csv(OUTPUT_CSV)

    # โหลดรายการที่เคยรันแล้ว (resume)
    done_map = load_done_map(OUTPUT_CSV)
    print(f"🔁 Resuming. Already have {len(done_map)} rows in {OUTPUT_CSV.name}" if done_map else "🆕 Starting fresh.")

    session = requests.Session()
    session.headers.update({"User-Agent": "eve-ocr-runner/2.2"})

    try:
        total = len(images)
        for i, img in enumerate(images, 1):
            prev = done_map.get(img.name)
            if prev and (prev.get("error", "").strip() == "") and (prev.get("text", "").strip() != ""):
                print(f"[{i}/{total}] {img.name}  ⏭️  (skip: already OK)")
                continue

            print(f"[{i}/{total}] {img.name}")
            text, err = ocr_one(session, img)

            if err:
                print(f"  ⚠️ {err}")
            else:
                preview = text.replace("\r", " ").replace("\n", " ")
                print(f"  ✅ {preview[:140]}{'...' if len(preview) > 140 else ''}")

            append_row(OUTPUT_CSV, {"filename": img.name, "text": text, "error": err})

            if i < total:
                time.sleep(SLEEP_BETWEEN_CALLS)

        print(f"\n✅ Done. Results saved to: {OUTPUT_CSV}")
    except KeyboardInterrupt:
        print("\n🛑 Interrupted. Partial results are already saved to CSV.")
    except Exception as e:
        print(f"\n💥 Unexpected error: {repr(e)}\nPartial results saved to CSV if any.")


🔁 Resuming. Already have 271 rows in ocr_results_2025-07-03.csv
[1/275] 2025-11-04_detail_0001.jpg  ⏭️  (skip: already OK)
[2/275] 2025-11-04_detail_0002.jpg  ⏭️  (skip: already OK)
[3/275] 2025-11-04_detail_0003.jpg  ⏭️  (skip: already OK)
[4/275] 2025-11-04_detail_0004.jpg  ⏭️  (skip: already OK)
[5/275] 2025-11-04_detail_0005.jpg  ⏭️  (skip: already OK)
[6/275] 2025-11-04_detail_0006.jpg  ⏭️  (skip: already OK)
[7/275] 2025-11-04_detail_0007.jpg  ⏭️  (skip: already OK)
[8/275] 2025-11-04_detail_0008.jpg  ⏭️  (skip: already OK)
[9/275] 2025-11-04_detail_0009.jpg  ⏭️  (skip: already OK)
[10/275] 2025-11-04_detail_0010.jpg  ⏭️  (skip: already OK)
[11/275] 2025-11-04_detail_0011.jpg  ⏭️  (skip: already OK)
[12/275] 2025-11-04_detail_0012.jpg  ⏭️  (skip: already OK)
[13/275] 2025-11-04_detail_0013.jpg  ⏭️  (skip: already OK)
[14/275] 2025-11-04_detail_0014.jpg  ⏭️  (skip: already OK)
[15/275] 2025-11-04_detail_0015.jpg  ⏭️  (skip: already OK)
[16/275] 2025-11-04_detail_0016.jpg  ⏭️  (ski